In [1]:
import pandas as pd
import numpy as np
import time

import seaborn
from matplotlib import pyplot

from sklearn.linear_model import LogisticRegression

In [2]:
county_crime_dataset = pd.read_csv('Index_Crimes_by_County_and_Agency__\
Beginning_1990.csv')

In [3]:
county_crime_dataset.head()

,County,Agency,Year,Months Reported,Index Total,Violent Total,Murder,Rape,Robbery,Aggravated Assault,Property Total,Burglary,Larceny,Motor Vehicle Theft,Region
0,Albany,Albany City PD,1990,NaN,6635.0,1052.0,9.0,82.0,386.0,575.0,5583.0,1884.0,3264.0,435.0,Non-New York City
1,Albany,Albany City PD,1991,NaN,7569.0,1201.0,11.0,71.0,487.0,632.0,6368.0,1988.0,3878.0,502.0,Non-New York City
2,Albany,Albany City PD,1992,NaN,7791.0,1150.0,8.0,77.0,467.0,598.0,6641.0,2246.0,3858.0,537.0,Non-New York City
3,Albany,Albany City PD,1993,NaN,7802.0,1238.0,6.0,59.0,481.0,692.0,6564.0,2063.0,4030.0,471.0,Non-New York City
4,Albany,Albany City PD,1994,NaN,8648.0,1380.0,13.0,79.0,542.0,746.0,7268.0,2227.0,4502.0,539.0,Non-New York City
